### Example for NEB calculation Cl$^-$+CH$_3$Cl using ASE+Conquest

In [ ]:
import os
import numpy as np
import matplotlib.pylab as plt

from ase import Atoms
from ase.io import read, write
from ase.visualize import view
from ase.constraints import FixAtoms
from ase.calculators.conquest import Conquest
from ase.calculators.emt import EMT

from ase.mep import NEB, NEBTools
from ase.optimize import BFGS

from cq_ase_external_lib import read_conquest

In [ ]:
%%bash 
ase --version

#### Define Conquest environment

In [ ]:
CQ_ROOT = '/Users/lioneltruflandier/CONQUEST-release-1.2'
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 2 '+CQ_ROOT+'/bin/Conquest'
os.environ['CQ_PP_PATH'] = CQ_ROOT+'/pseudo-and-pao/'
os.environ['CQ_GEN_BASIS_CMD'] = CQ_ROOT+'/bin/MakeIonFiles'

#### Directory for storing calculation files

In [ ]:
working_directory = 'cq_example_CH3Cl_neb_ase'

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

####  Read intial and final state from `ASE` trajectory files

> Note that the intial and final state have already been optimized

In [ ]:
initial = read('./struct/initial.traj')
view(initial)

In [ ]:
final = read('./struct/final.traj')
view(final)

####  (Re-)optimized initial and final state with Conquest

> PAO setup using small basis size

In [ ]:
basis = {'C' : { 'gen_basis'             : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann'},
         'Cl': { 'gen_basis'             : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann'},
         'H' : { 'gen_basis'             : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann'}
        }

> Conquest calculator setup for positions optimisation ; note the overall charge of the system

In [ ]:
cutoff  =  90.0
kpoints = [1,1,1]
fxc     = 'PBE'

conquest_flags = {'General.NetCharge' : 1.0}

# Flags for atomic positions optimisation
conquest_flags.update({'AtomMove.TypeOfRun'   : 'sqnm',  # optimization algorithm
                       'AtomMove.MaxForceTol' :  1e-3,   # max Force component in Ha/bohr                       
                       'AtomMove.ReuseDM'     :  True,    
                       'AtomMove.AppendCoords':  True,
                       })    

calc = Conquest(directory      = working_directory,
                grid_cutoff    = cutoff,
                self_consistent= True,
                xc    = fxc,
                basis = basis,
                kpts  = kpoints,
                nspin = 1,
                **conquest_flags)

> Constrain the position of C atom in the cell for initial state

In [ ]:
mask = [atom.symbol == 'C'  for atom in initial]
constraint = FixAtoms(mask=mask)
initial.set_constraint(constraint)

> Initial state optimisation

In [ ]:
initial.calc   = calc
initial_energy = initial.get_potential_energy(apply_constraint=True)

> Initial state geometry

In [ ]:
atomic_order = ['H','C','Cl']
initial_geom = read_conquest(working_directory+'/coord_next.dat', fractional=True, atomic_order=atomic_order)

In [ ]:
view(initial_geom)
write(working_directory+'/Cl-CH3Cl.traj', initial_geom)

> Constrain the position of C atom in the cell for final state

In [ ]:
mask = [atom.symbol == 'C'  for atom in final]
constraint = FixAtoms(mask=mask)
final.set_constraint(constraint)

> Final state optimisation

In [ ]:
final.calc   = calc
final_energy = final.get_potential_energy(apply_constraint=True)

> Final state geometry

In [ ]:
atomic_order = ['H','C','Cl']
final_geom = read_conquest(working_directory+'/coord_next.dat', fractional=True, atomic_order=atomic_order)

In [ ]:
view(final_geom)
write(working_directory+'/ClCH3-Cl.traj', final)

#### Prepare NEB calculation

> Load initial and final state

In [ ]:
initial = read(working_directory+'/Cl-CH3Cl.traj')
final   = read(working_directory+'/ClCH3-Cl.traj')

> Create constraints if any

In [ ]:
mask = [atom.symbol == 'C'  for atom in initial]
constraint = FixAtoms(mask=mask)

> Setup calculator parameters for NEB ; standard SCF calculations => don't forget to delete optimisation parameters 

In [ ]:
cutoff  =  90.0
kpoints = [1,1,1]
fxc     = 'PBE'
conquest_flags = {'General.NetCharge' : 1.0}

> Setup initial image calculator

In [ ]:
initial.calc = Conquest(directory = working_directory,
                   grid_cutoff    = cutoff,
                   self_consistent= True,
                   xc    = fxc,
                   basis = basis,
                   kpts  = kpoints,
                   nspin = 1,
                   scf_tolerance = 1e-7, 
                   **conquest_flags)

> Backup first image in a list

In [ ]:
images = [initial]

> Generate 3 intermediate images and calculators

In [ ]:
for i in range(3):
    image      = initial.copy()
    image.calc = Conquest(directory = working_directory,
                 grid_cutoff    = cutoff,
                 self_consistent= True,
                 xc    = fxc,
                 basis = basis,
                 kpts  = kpoints,
                 nspin = 1,
                 scf_tolerance = 1e-7, 
                 **conquest_flags)
  
    image.set_constraint(constraint)
    images.append(image)

> Setup final image calculator

In [ ]:
final.calc = Conquest(directory = working_directory,
                 grid_cutoff    = cutoff,
                 self_consistent= True,
                 xc    = fxc,
                 basis = basis,
                 kpts  = kpoints,
                 nspin = 1,
                 scf_tolerance = 1e-7, 
                 **conquest_flags)

> Backup final image

In [ ]:
images.append(final)

#### Launch the NEB calculation

> Generate NEB ASE object and interpolate intermediate images

In [ ]:
neb = NEB(images)
neb.interpolate(apply_constraint=True)

> Setup optimisation method

In [ ]:
qn = BFGS(neb, trajectory='neb.traj')

> Run the NEB optimisation

In [ ]:
qn.run(fmax=1.0)

#### Perform a last SCF calculation for each converged images (can't be avoid for now...)

In [ ]:
energies = [ ]
for image in images:
    energies.append(image.get_potential_energy())
#print(energies)

#### Extract data and plot using `NEBtools`

In [ ]:
nebtools = NEBTools(images)

# Get the calculated barrier and the energy change of the reaction.
Ef, dE = nebtools.get_barrier()

# Get the actual maximum force at this point in the simulation.
max_force = nebtools.get_fmax() ; print('max_force = ',max_force)

# Create a figure like that coming from ASE-GUI.
fig = nebtools.plot_band()
#fig.savefig('diffusion-barrier.png')

# Create a figure with custom parameters.
#fig = plt.figure(figsize=(5.5, 4.0))
#ax = fig.add_axes((0.15, 0.15, 0.8, 0.75))
#nebtools.plot_band(ax)
#fig.savefig('diffusion-barrier.png')



In [ ]:
# to visualise the trajectory type $ase gui neb.traj@:5